In [1]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
import datetime


In [2]:
# Importation des librairies
import os
import time
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import product
import random
from random import seed as random_seed

from tqdm import tqdm
import plotly.graph_objects as go


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Configuration pour éviter les avertissements
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import plotly.offline as py 
py.init_notebook_mode(connected=True) # this code, allow us to work with offline plotly version
import plotly.graph_objs as go # it's like "plt" of matplot
import plotly.tools as tls # It's useful to we get some tools of plotly


In [3]:
# Authentification
credential = DefaultAzureCredential()

# Client pour gérer les groupes de ressources
resource_client = ResourceManagementClient(credential, subscription_id)

# Vérifier si le groupe de ressources existe
try:
    resource_client.resource_groups.get(resource_group)
    print(f"Le groupe de ressources '{resource_group}' existe déjà.")
except Exception:
    print(f"Création du groupe de ressources '{resource_group}'...")
    resource_client.resource_groups.create_or_update(
        resource_group_name=resource_group,
        parameters={"location": location},
    )

Création du groupe de ressources 'ds-ml-eus-data'...


In [5]:
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group)

In [6]:
import datetime

basic_workspace_name = "mlw-basic-prod-" + datetime.datetime.now().strftime(
    "%Y%m%d%H%M"
)

ws_basic = Workspace(
    name=basic_workspace_name,
    location="eastus",
    display_name="Basic workspace-example",
    description="This example shows how to create a basic workspace",
    hbi_workspace=False,
    tags=dict(purpose="demo"),
)

ws_basic = ml_client.workspaces.begin_create(ws_basic).result()
print(ws_basic)

The deployment request mlw-basic-prod-202502041227-2708866 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F701cba3b-fdc2-498c-8f7c-16ebfcf45e76%2FresourceGroups%2Fds-ml-eus-data%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fmlw-basic-prod-202502041227-2708866
Creating Key Vault: (mlwbasickeyvault46d1710b  ) ..  Done (21s)
Creating Log Analytics Workspace: (mlwbasiclogalytifc475b58  )  Done (22s)
Creating Storage Account: (mlwbasicstorage9fa679174  )  Done (23s)
Creating AzureML Workspace: (mlw-basic-prod-202502041227  ) ..  Done (22s)
Creating Application Insights: (mlwbasicinsightsc84e055e  )  Done (25s)
Total time : 50s



allow_roleassignment_on_rg: true
application_insights: /subscriptions/701cba3b-fdc2-498c-8f7c-16ebfcf45e76/resourceGroups/ds-ml-eus-data/providers/Microsoft.insights/components/mlwbasicinsightsc84e055e
description: This example shows how to create a basic workspace
discovery_url: https://eastus.api.azureml.ms/discovery
display_name: Basic workspace-example
enable_data_isolation: false
hbi_workspace: false
id: /subscriptions/701cba3b-fdc2-498c-8f7c-16ebfcf45e76/resourceGroups/ds-ml-eus-data/providers/Microsoft.MachineLearningServices/workspaces/mlw-basic-prod-202502041227
identity:
  principal_id: 5c3670af-f979-4395-ac7b-44812eda994b
  tenant_id: f6449130-f985-4f70-b860-6b37d30e54fc
  type: system_assigned
key_vault: /subscriptions/701cba3b-fdc2-498c-8f7c-16ebfcf45e76/resourceGroups/ds-ml-eus-data/providers/Microsoft.Keyvault/vaults/mlwbasickeyvault46d1710b
location: eastus
managed_network:
  isolation_mode: disabled
  outbound_rules: []
mlflow_tracking_uri: azureml://eastus.api.azureml

In [8]:
from azure.ai.ml.entities import AzureBlobDatastore

store = AzureBlobDatastore(
    name="mldata_store",
    description="first projet",
    account_name="mlprojecstorageb7d68ab92",
    container_name="azureml"
)

ml_client.create_or_update(store)

Found the config file in: .\config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'mldata_store', 'description': 'first projet', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/701cba3b-fdc2-498c-8f7c-16ebfcf45e76/resourceGroups/ds-ml-eus-data/providers/Microsoft.MachineLearningServices/workspaces/mlw-basic-prod-202502041227/datastores/mldata_store', 'Resource__source_path': '', 'base_path': 'c:\\Users\\Rajeeth\\Desktop\\AzureML - Copie\\src', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000002AB125B5070>, 'credentials': <azure.ai.ml.entities._credentials.NoneCredentialConfiguration object at 0x000002AB125B3F70>, 'container_name': 'azureml', 'account_name': 'mlprojecstorageb7d68ab92', 'endpoint': 'core.windows.net', 'protocol': 'https'})

In [ ]:
from azure.ai.ml.entities import Data

# Define the Data asset object
my_data = Data(
    path="cs-training.csv",
    type="uri_file",
    description="My images. Version includes data through to 2023-Jan-08.",
    name="risk_credits_dataset",
    version="1",
)

# Create the data asset in the workspace
ml_client.data.create_or_update(my_data)